In [1]:
import os
import torchvision
import torch
import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.display import display, Image, clear_output
import ipywidgets as widgets
import threading as thread
import time
import traitlets
import dataset
from uuid import uuid1
import json
import glob
from datetime import datetime
HOME = os.getcwd()
print(HOME)

import ultralytics
ultralytics.checks()

from ultralytics import YOLO
from PIL import Image
import requests

model = YOLO('yolo11q.pt')
model2 = YOLO('yolo11s.pt')

%cd {HOME}
!mkdir datasets
%cd {HOME}/datasets

from roboflow import Roboflow

ROBOFLOW_API_KEY = ('14W8hOR0bWFloIDQecFa')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project2 = rf.workspace("screw-sorting-project").project("side-screw-sorter")
version2 = project2.version(4)
dataset2 = version2.download("yolov11")

project = rf.workspace("screw-sorting-project").project("screw-sorting")
version = project.version(4)
dataset = version.download("yolov11")


Ultralytics 8.3.44 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6064MiB)
Setup complete ✅ (12 CPUs, 15.6 GB RAM, 87.4/915.8 GB disk)
/home/independent-study/Desktop
mkdir: cannot create directory ‘datasets’: File exists


/home/independent-study/independentstudyethan/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/independent-study/Desktop/datasets
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...


In [2]:
import cv2
import supervision as sv
import numpy as np
from threading import Thread
from queue import Queue
import torch
import inference
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

stop_event = thread.Event()
stop_cap_event = thread.Event()
stop_cap1_event = thread.Event()

# Select the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize Model
model_id = project.id.split("/")[1] + "/" + dataset.version
model_id2 = project2.id.split("/")[1] + "/" + dataset2.version
model = inference.get_model(model_id, '14W8hOR0bWFloIDQecFa')
model2 = inference.get_model(model_id2, '14W8hOR0bWFloIDQecFa')

# Initialize webcam
cap = cv2.VideoCapture(0)  # First webcamcombined_labels_array = sorted(combined_labels_array2, key=lambda x: x[2])
cap2 = cv2.VideoCapture(2)  # Second webcam
focus = 0  # min: 0, max: 255, increment:5
cap2.set(28, focus) 

# Image widgets for displaying frames
image_widget = widgets.Image(format='jpeg', width=640, height=360)
image_widget2 = widgets.Image(format='jpeg', width=640, height=360)
display(image_widget, image_widget2)

# Queue to store frames
frame_queue = Queue(maxsize=5)
frame_queue2 = Queue(maxsize=5)



# Frame capture function for the first webcam
def capture_frames():
    cap.set(cv2.CAP_PROP_FOCUS, 50)
    while not stop_cap_event.is_set():
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image from webcam 1.")
            break
        if not frame_queue.full():
            frame_queue.put(frame)
    cap.release()

# Frame capture function for the second webcam
def capture_frames1():
    cap2.set(cv2.CAP_PROP_FOCUS, 50)
    while not stop_cap1_event.is_set():
        ret, frame2 = cap2.read()
        frame2 = cv2.flip(frame2, 1)
        if not ret:
            print("Failed to capture image from webcam 2.")
            break
        if not frame_queue2.full():
            frame_queue2.put(frame2)
    cap2.release()

# Frame processing and inference function
xCoordinate = 0
xCoordinate2 = 0
labels2 = [2]
combinedLabelsSide = np.array([])
combinedLabelsSide2 = np.array([])
combined_labels_array = []  # Initialize a 2D array to store the results
combined_labels_array2 = []  # Initialize a 2D array to store the results
complete_labels_array =[]
filtered_detections =[]
def process_and_display():
    global xCoordinate
    global xCoordinate2
    global labels2
    global combinedLabelsSide
    global combinedLabelsSide2
    global combined_labels_array
    global combined_labels_array2
    global complete_labels_array
    global filtered_detections
    print("Thread started")
    while not stop_event.is_set():
        # Process first webcam frames
        if not frame_queue2.empty():
            frame2 = frame_queue2.get()
            results = model2.infer(frame2, confidence=0.4, overlap=0.3)[0]
            detections2 = sv.Detections.from_inference(results)
            if detections2.xyxy.size != 0:
                combined_labels_array = []  # Initialize a 2D array to store the results
                for i in range(len(detections2.xyxy)):
                    labels2 = detections2.data.get('class_name', None)
                    xCoordinate2 = (detections2.xyxy[i][2] + detections2.xyxy[i][0]) / 2
                    for n in range(len(detections2.xyxy)):
                        if n != i:
                            other_x_coordinate = (detections2.xyxy[n][2] + detections2.xyxy[n][0]) / 2
                            if abs(xCoordinate2 - other_x_coordinate) <= 7:
                                combinedLabelsSide = [xCoordinate2, labels2[i], labels2[n]]
                                combinedLabelsSide.sort(key=repr)  # Sort combinedLabelsSide
                                combined_labels_array.append(combinedLabelsSide)  # Add to 2D array
                                combined_labels_array = sorted(combined_labels_array, key=lambda x: x[2])
                                n = len(detections2.xyxy)  # Break inner loop
            # Annotate boxes and labels
            box_annotator = sv.BoxAnnotator()
            #label_annotator = sv.LabelAnnotator()
            annotated_frame2 = box_annotator.annotate(scene=frame2, detections=detections2)
            #annotated_frame2 = label_annotator.annotate(scene=annotated_frame2, detections=detections2)
            _, jpeg2 = cv2.imencode('.jpg', annotated_frame2)
            image_widget2.value = jpeg2.tobytes()


        #run = 0
        if not frame_queue.empty():
            frame = frame_queue.get()
            results = model.infer(frame, confidence=0.4, overlap=0.3)[0]
            detections = sv.Detections.from_inference(results)
            if detections.xyxy.size != 0:
                combined_labels_array2 = []  # Initialize a 2D array to store the results
                for i in range(len(detections.xyxy)):
                    labels2 = detections.data.get('class_name', None)
                    xCoordinate2 = (detections.xyxy[i][2] + detections.xyxy[i][0]) / 2
                    combinedLabelsSide2 = [labels2[i], xCoordinate2]
                    combined_labels_array2.append(combinedLabelsSide2)  # Add to 2D array
                    combined_labels_array2.sort(key=lambda x: x[1])
            # Annotate boxes and labels
            box_annotator = sv.BoxAnnotator()
            #label_annotator = sv.LabelAnnotator()
            annotated_frame = box_annotator.annotate(scene=frame, detections=detections)
            #annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections)
            # Update the widget
            _, jpeg = cv2.imencode('.jpg', annotated_frame)
            image_widget.value = jpeg.tobytes()   

        if((len(combined_labels_array))>0):
            filtered_detections = []
            for detection in combined_labels_array:
                if not filtered_detections or abs(detection[2] - filtered_detections[-1][2]) >= 6:
                    filtered_detections.append(detection)

        # print("legnth of 1d array:", len(combined_labels_array2))
        # print("legnth of 2d array:", len(filtered_detections))
        if len(combined_labels_array2) == len(filtered_detections):
            print("Detections:")
            for i in range(len(combined_labels_array2)):  # Use range to iterate by index
                # Append a tuple or list to complete_labels_array
                complete_labels_array.append([
                    filtered_detections[i][0],  # First element from filtered_detections
                    filtered_detections[i][1],  # Second element from filtered_detections
                    combined_labels_array2[i][0],  # First element from combined_labels_array2
                    filtered_detections[i][2]   # Third element from filtered_detections
                ])
            for detection in complete_labels_array:
                    print(detection)
            complete_labels_array = []


    frame_queue2.empty()
    frame_queue.empty()
    # Release cameras when done
    print("thread stopped")

# Start the capture and processing threads
# capture_thread = Thread(target=capture_frames)
# capture_thread1 = Thread(target=capture_frames1)
# process_thread = Thread(target=process_and_display)

def start_process_thread():
    global process_thread
    stop_event.clear()  # Clear the stop event to allow the thread to run
    process_thread = Thread(target=process_and_display)
    process_thread.start()

def start_capture_thread():
    global capture_thread
    stop_cap_event.clear()  # Clear the stop event to allow the thread to run
    capture_thread = Thread(target=capture_frames)
    capture_thread.start()

def start_capture1_thread():
    global capture_thread1
    stop_cap1_event.clear()  # Clear the stop event to allow the thread to run
    capture_thread1 = Thread(target=capture_frames1)
    capture_thread1.start()


def stop_process_thread():
    stop_event.set()  # Signal the thread to stop
    process_thread.join()     # Wait for the thread to finish

def stop_capture_thread():
    stop_cap_event.set()  # Signal the thread to stop
    capture_thread.join()     # Wait for the thread to finish

def stop_capture1_thread():
    stop_cap1_event.set()  # Signal the thread to stop
    capture_thread1.join()     # Wait for the thread to finish

start_capture_thread() 
start_capture1_thread()
time.sleep(3)   # Let it run for a while

start_process_thread()  # Start the thread
time.sleep(10)   # Let it run for a while
stop_process_thread()   # Stop the thread
# time.sleep(2)   # Let it run for a while

# start_process_thread()  # Restart the thread
# time.sleep(3)   # Let it run for a while
# stop_process_thread()   # Stop it again
# time.sleep(2)   # Let it run for a while

# start_process_thread()  # Restart the thread
# time.sleep(3)   # Let it run for a while
# stop_process_thread()   # Stop it again
# time.sleep(2)   # Let it run for a while



stop_capture_thread()
stop_capture1_thread()
print("all threads terminated")

# To stop the loops, set is_running = False in your notebook


[04/11/25 10:14:53] WARNING  Your inference package version 0.29.2 is out of date! Please upgrade to ]8;id=13062;file:///home/independent-study/independentstudyethan/lib/python3.12/site-packages/inference/core/__init__.py\__init__.py]8;;\:]8;id=827709;file:///home/independent-study/independentstudyethan/lib/python3.12/site-packages/inference/core/__init__.py#41\41]8;;\
                             version 0.46.4 of inference for the latest features and bug fixes by                  
                             running `pip install --upgrade inference`.                                            

Using device: cuda


Image(value=b'', format='jpeg', height='360', width='640')

Image(value=b'', format='jpeg', height='360', width='640')

Thread started
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
Detections:
D

In [1]:
pip install pyserial


  Using cached pyserial-3.5-py2.py3-none-any.whl.metadata (1.6 kB)
Using cached pyserial-3.5-py2.py3-none-any.whl (90 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2

# Initialize the camera
cap = cv2.VideoCapture(0)  # 0 is the index for the default camera

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open the camera.")
    exit()

# Set camera properties
focus = 10 # min: 0, max: 255, increment:5
if not cap.set(28, focus):
    print("Focus control is not supported by this camera.")
else:
    print(f"Focus set to {focus}")

# Capture frames
while True:
    cap.set(28, focus)
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Display the frame
    cv2.imshow("Webcam", frame)

    # Break the loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


Focus control is not supported by this camera.
